In [ ]:
from fastapi import FastAPI, HTTPException, Request
import pandas as pd
import xgboost as xgb
import os
import warnings
import logging
from slowapi import Limiter
from slowapi.middleware import SlowAPIMiddleware
from fastapi.responses import JSONResponse
from pydantic import BaseModel
import uvicorn

# Suppress XGBoost warnings
warnings.filterwarnings('ignore', category=UserWarning, module='xgboost')

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger(__name__)

# Rate Limiting Setup (to prevent abuse)
limiter = Limiter(key_func=lambda: "global", default_limits=["100/minute"])
app = FastAPI(
    title="Fraud Detection API",
    version="2.0",
    description="A real-time fraud detection API using XGBoost.",
    docs_url="/docs",
    openapi_url="/openapi.json",
    redoc_url="/redoc",
)
app.state.limiter = limiter
app.add_middleware(SlowAPIMiddleware)

# Exception handler for rate limiting
@app.exception_handler(Exception)
async def rate_limit_handler(request: Request, exc: Exception):
    return JSONResponse(status_code=429, content={"detail": "Too many requests. Please try again later."})

# Get the absolute path of the current script
BASE_DIR = os.path.dirname(os.path.abspath(__file__))

# Load Fraud Detection Model
MODEL_PATH = os.path.join(BASE_DIR, "fraud_detection_model.json")
if not os.path.exists(MODEL_PATH):
    raise SystemExit(f"Error: Model file not found! Ensure it exists at: {MODEL_PATH}")

try:
    model = xgb.XGBClassifier()
    model.load_model(MODEL_PATH)
    logger.info("Fraud detection model loaded successfully.")
except Exception as e:
    raise SystemExit(f"Error: Failed to load model: {str(e)}")

### TRANSACTION REQUEST MODEL ###
class TransactionInput(BaseModel):
    trans_date: str
    unix_time: float
    amt: float
    merch_lat: float
    merch_long: float
    customer_num_trans_1_day: int
    customer_num_trans_7_day: int
    customer_num_trans_30_day: int
    trans_time_is_night: int
    trans_time_day: int
    trans_date_is_weekend: int
    customer_avg_amount_1_day: float
    customer_avg_amount_7_day: float
    customer_avg_amount_30_day: float
    merchant_num_trans_1_day: int
    merchant_num_trans_7_day: int
    merchant_num_trans_30_day: int
    merchant_risk_1_day: float
    merchant_risk_7_day: float
    merchant_risk_30_day: float
    merchant_risk_90_day: float
    transaction_hour: int
    customer_avg_transaction_amount: float
    merchant_fraud_rate: float
    merchant_high_risk_flag: int
    hourly_fraud_rate: float
    is_high_risk_hour: int

### FRAUD PREDICTION ENDPOINT ###
@app.post("/predict", tags=["Fraud Detection"])
@limiter.limit("5/minute")  # Limits requests to 5 per minute
async def predict_fraud(transaction: TransactionInput, request: Request):
    try:
        # Convert input to DataFrame
        input_df = pd.DataFrame([transaction.dict()])
        input_df['trans_date'] = pd.to_datetime(input_df['trans_date']).astype('int64') // 10**9

        # Make fraud prediction
        fraud_probability = float(model.predict_proba(input_df)[:, 1][0])
        logger.info(f"Fraud Probability: {fraud_probability}")

        return {"fraud_probability": fraud_probability}

    except Exception as e:
        logger.error(f"Prediction Error: {str(e)}")
        raise HTTPException(status_code=500, detail=f"Prediction error: {str(e)}")


if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)
